## Top Ten Hashtags Alternative

In [ ]:
# run python TweetRead.py > data.txt then run this file

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext

import time
import pandas as pd
from IPython import display
import matplotlib.pyplot as plt

sc = SparkContext()
ssc = StreamingContext(sc, 1 )
sqlContext = SQLContext(sc)
lines = ssc.socketTextStream("localhost", 9999)
( lines.flatMap(lambda line: line.split(" "))
    .filter( lambda word: word.lower().startswith("#") )
    .map(lambda word: (word, 1))
    .reduceByKey(lambda x, y: x + y)
    .foreachRDD( lambda rdd: rdd.toDF()) )

ssc.start()

In [ ]:
filepath = "data.txt"
count = 0
while True:
    time.sleep( 10 )    
    count += 1

    lines = []
    with open(filepath) as fp:
        line = fp.readline()
        cnt = 1
        while line:
            lines.append(line.strip())
            line = fp.readline()        
            cnt += 1

    pdf = pd.DataFrame(lines, columns=["tag"])
   
    hashtags = []
    for i in pdf.items():
        for j in list(i[1]):        
            [hashtags.append(word) for word in j.split(" ") if word.startswith("#")]

    df = pd.DataFrame(hashtags, columns=["hashtag"])
    display.clear_output(wait=True)
    print("seconds running: {}".format(count * 10))
    print("hashtag count: ", len(df["hashtag"]))
    if len(df.index) > 0:
        df["hashtag"].value_counts().head(10).plot(kind="bar")
        plt.title("Top Ten Hashtags")
        plt.show()